In [1]:
!pip install tweepy

In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from datetime import datetime
import json

In [3]:
consumer_key = 'RETIRADO POR SEGURANÇA'

In [4]:
consumer_secret = 'RETIRADO POR SEGURANÇA'

In [6]:
acess_token = 'RETIRADO POR SEGURANÇA'

In [7]:
acess_token_secret = 'RETIRADO POR SEGURANÇA'

In [8]:
auth = OAuthHandler(consumer_key, consumer_secret)

In [9]:
auth.set_access_token(acess_token, acess_token_secret)

In [11]:
# Criando uma classe para capturar os stream de dados do Twitter
# armazenar no Mongodb
class MyListener(StreamListener):
    def on_data(self, dados):
        tweet = json.loads(dados)
        created_at = tweet['created_at']
        id_str = tweet['id_str']
        text = tweet['text']
        obj = {'created_at': created_at,
               'id_str': id_str,
              'text': text,}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [12]:
mylistener = MyListener()

In [13]:
mystream = Stream(auth, listener = mylistener)

In [14]:
from pymongo import MongoClient

In [15]:
client = MongoClient('localhost', 27017)

In [16]:
db = client.twitterdb

In [17]:
col = db.tweets

In [18]:
keywords = ['Big Data', 'Python', 'Data Mining', 'Data Science']

In [21]:
mystream.filter(track=keywords)

{'created_at': 'Thu Sep 13 23:00:52 +0000 2018', 'id_str': '1040374785908199425', 'text': 'A nivel global, empresas de todos los tamaños están identificando las áreas clave en donde la inteligencia artifici… https://t.co/14wGDmAVac', '_id': ObjectId('5b9aec2471f8d21a9c95819a')}
{'created_at': 'Thu Sep 13 23:00:55 +0000 2018', 'id_str': '1040374798453493766', 'text': 'RT @bberry_jia: For @fecompendium s blue challenge ^^ I JUST LOVE THIS BLUEBERRY HEAD JAHDKSBD PYTHON LOVE ME BACK PLS 😭😭💙💙💙💙 #FireEmblem #…', '_id': ObjectId('5b9aec2771f8d21a9c95819b')}
{'created_at': 'Thu Sep 13 23:00:58 +0000 2018', 'id_str': '1040374807664242688', 'text': "@realDonaldTrump \n\nManafort flipped. Just like one of those Jacob ladder toys. \n\nWho's next? Donnie? Will Pauly fli… https://t.co/CNGfXb5TcQ", '_id': ObjectId('5b9aec2971f8d21a9c95819c')}
{'created_at': 'Thu Sep 13 23:01:07 +0000 2018', 'id_str': '1040374847916920834', 'text': "RT @hazybluedot: Finally: 'master/slave' terminology removed from Py

KeyboardInterrupt: 

In [22]:
mystream.disconnect()

In [23]:
col.find_one()

{'_id': ObjectId('5b9aec2471f8d21a9c95819a'),
 'created_at': 'Thu Sep 13 23:00:52 +0000 2018',
 'id_str': '1040374785908199425',
 'text': 'A nivel global, empresas de todos los tamaños están identificando las áreas clave en donde la inteligencia artifici… https://t.co/14wGDmAVac'}

In [25]:
dataset = [{'created_at': item['created_at'], 'text': item['text'],} for item in col.find()]

In [26]:
import pandas as pd

In [27]:
df = pd.DataFrame(dataset)

In [28]:
df.head()

,created_at,text
0,Thu Sep 13 23:00:52 +0000 2018,"A nivel global, empresas de todos los tamaños ..."
1,Thu Sep 13 23:00:55 +0000 2018,RT @bberry_jia: For @fecompendium s blue chall...
2,Thu Sep 13 23:00:58 +0000 2018,@realDonaldTrump \n\nManafort flipped. Just li...
3,Thu Sep 13 23:01:07 +0000 2018,RT @hazybluedot: Finally: 'master/slave' termi...
4,Thu Sep 13 23:01:07 +0000 2018,RT @PyDataCba: 💣💥 PyData Córdoba \n▶30th Septe...


In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [30]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(df.text)

In [31]:
# Contando as quantas vezes as palavras aparecem
word_count = pd.DataFrame(cv.get_feature_names(), columns=['word'])
word_count['count'] = count_matrix.sum(axis=0).tolist()[0]
word_count = word_count.sort_values('count', ascending=False).reset_index(drop=True)
word_count[:50]

,word,count
0,https,17
1,co,16
2,data,12
3,rt,11
4,the,8
5,python,8
6,for,7
7,big,7
8,to,7
9,and,6
